In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.nn as nn
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import Tensor, optim, cuda
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import csv
import os

from data.data import DeClareDataset
from model.deClare import DeClareModel

In [2]:
torch.__version__

'1.0.0'

# Load Datasets

In [3]:
SNOPES_LOC = "./Datasets/Snopes/snopes.tsv"
#consists of rumors analyzed on the Snopes website along with their credibility labels (true or false), 
#sets of reporting articles, and their respective web sources

POLITIFACT_LOC = "./Datasets/PolitiFact/politifact.tsv"

glove_data_file = "./Glove/glove.6B.100d.txt"

In [4]:
snopes = DeClareDataset(SNOPES_LOC, glove_data_file)

Successfully read news data from ./Datasets/Snopes/snopes.tsv
Number of articles = 29242
Number of claims = 4341
Using pre-built vocabulary


## Preliminary Analysis

In [5]:
batch_size = 4
nb_lstm_units = 64

In [6]:
dataloader = DataLoader(snopes, batch_size)

In [7]:
declare = DeClareModel(snopes.initial_embeddings, snopes.claim_source_vocab_size, 
                       snopes.article_source_vocab_size, nb_lstm_units, batch_size)

In [8]:
for data_sample in dataloader:
    temp = declare(data_sample[0], data_sample[1], data_sample[2], data_sample[3], data_sample[4], data_sample[5])
    break

/home/manu/Projects/prod/DeClare/model/deClare.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attentions = F.softmax(claim_article_concat)
/home/manu/Software/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([4, 1])

In [ ]:
temp.shape

In [11]:
snopes.news_df['Claim'].iloc[0]

'best buy chain eschewing use word christmas 2006 holiday print advertising'

In [12]:
snopes.news_df['Article'].str.split().str.len().min()

21

In [13]:
snopes._vec('there')

/home/manu/Projects/prod/DeClare/data/data.py:114: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return self.glove_df.loc[w].as_matrix()


array([-0.084215 ,  0.69597  ,  0.28383  , -0.22497  , -0.55923  ,
        0.21196  , -0.15175  ,  0.31601  ,  0.24803  , -0.36385  ,
        0.2252   ,  0.34109  ,  0.61438  ,  0.08318  ,  0.74894  ,
       -0.38785  , -0.27211  ,  0.2656   , -0.66332  ,  0.45571  ,
        0.34949  ,  0.38635  ,  0.25707  , -0.60101  , -0.047292 ,
       -0.41198  ,  0.38246  , -0.52151  ,  0.037757 , -0.42492  ,
       -0.36352  , -0.037377 ,  0.31263  ,  0.19709  ,  0.008142 ,
        0.53871  , -0.074505 ,  0.31395  ,  0.35129  , -0.39005  ,
       -0.5474   , -0.11395  ,  0.076668 , -0.61069  ,  0.15894  ,
       -0.33043  ,  0.75967  , -0.52289  , -0.38957  , -0.71164  ,
        0.28724  , -0.35683  ,  0.050529 ,  1.3392   , -0.16064  ,
       -2.9236   ,  0.17524  , -0.42109  ,  1.5235   ,  0.85181  ,
       -0.47563  ,  1.1225   , -0.48463  ,  0.36458  ,  0.97809  ,
       -0.2227   ,  0.88791  ,  0.068738 ,  0.22557  ,  0.026459 ,
       -0.032799 , -0.40426  ,  0.14579  , -0.4535   ,  0.4775

In [14]:
a = torch.ones(5, 2, 4)
b = torch.randn(5, 4)
#c shape (5, 2, 8)

In [15]:
a.mean(1)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [17]:
c = torch.cat([a,b.unsqueeze(1).expand(-1, 2, -1)], 2)

In [18]:
a, b

(tensor([[[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.]]]), tensor([[ 1.2066, -0.0813, -0.3177,  0.6673],
         [ 0.3020,  0.8963,  2.3230, -0.6914],
         [-1.1176,  0.1443, -1.1646, -1.4895],
         [ 0.9507,  1.2500, -0.6413,  0.4825],
         [ 0.4117, -0.4153,  1.0821,  0.4459]]))

In [19]:
c

tensor([[[ 1.0000,  1.0000,  1.0000,  1.0000,  1.2066, -0.0813, -0.3177,
           0.6673],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.2066, -0.0813, -0.3177,
           0.6673]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000,  0.3020,  0.8963,  2.3230,
          -0.6914],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  0.3020,  0.8963,  2.3230,
          -0.6914]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000, -1.1176,  0.1443, -1.1646,
          -1.4895],
         [ 1.0000,  1.0000,  1.0000,  1.0000, -1.1176,  0.1443, -1.1646,
          -1.4895]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000,  0.9507,  1.2500, -0.6413,
           0.4825],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  0.9507,  1.2500, -0.6413,
           0.4825]],

        [[ 1.0000,  1.0000,  1.0000,  1.0000,  0.4117, -0.4153,  1.0821,
           0.4459],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  0.4117, -0.4153,  1.0821,
           0.4459]]])